In [6]:
import pandas as pd
import os

# Configuração inicial
caminho = 'C:\\Users\\Edmilson\\Gama_Workspace2\\ABMS-WP'

# Carregar o arquivo já processado com comportamentos
arquivo_df = pd.read_csv(
    os.path.join(caminho, 'includes\\Tabela_consumidores_Itapua_com_setor_e_comportamento.csv'), 
    sep=';'
)

# 1. Verificar e limpar dados
print("\nVerificando dados nulos:")
print(arquivo_df.isnull().sum())

# Remover quaisquer linhas com valores nulos restantes
arquivo_df = arquivo_df.dropna()

# 2. Análise por setor
# Criar uma tabela resumo por setor
resumo_setores = arquivo_df.groupby('CD_SETOR').agg(
    total_consumidores=('SK_MATRICULA', 'count'),
    ambientalistas=('TP_COMPORTAMENTO', lambda x: (x == 'AMBIENTALISTA').sum()),
    moderados=('TP_COMPORTAMENTO', lambda x: (x == 'MODERADO').sum()),
    perdularios=('TP_COMPORTAMENTO', lambda x: (x == 'PERDULARIO').sum()),
    media_consumo_diario=('NN_CONSUMO_DIARIO', 'mean')
).reset_index()

# Calcular percentuais
resumo_setores['%_ambientalistas'] = (resumo_setores['ambientalistas'] / resumo_setores['total_consumidores']) * 100
resumo_setores['%_moderados'] = (resumo_setores['moderados'] / resumo_setores['total_consumidores']) * 100
resumo_setores['%_perdularios'] = (resumo_setores['perdularios'] / resumo_setores['total_consumidores']) * 100

# Ordenar por maior percentual de perdulários
resumo_setores = resumo_setores.sort_values(by='%_perdularios', ascending=False)

# 3. Salvar o relatório em CSV
caminho_relatorio = os.path.join(caminho, 'includes\\Relatorio_Comportamento_Por_Setor.csv')
resumo_setores.to_csv(caminho_relatorio, sep=';', index=False, float_format='%.2f')

# 4. Visualizar resultados
print("\nTop 5 setores com mais perdulários:")
print(resumo_setores.head())

print("\nEstatísticas gerais:")
print(f"Total de setores analisados: {len(resumo_setores)}")
print(f"Média de consumidores por setor: {resumo_setores['total_consumidores'].mean():.1f}")
print(f"Setor com mais perdulários: {resumo_setores.iloc[0]['CD_SETOR']} ({resumo_setores.iloc[0]['%_perdularios']:.1f}%)")

# 5. Gerar também um resumo geral
resumo_geral = {
    'total_consumidores': arquivo_df['SK_MATRICULA'].nunique(),
    'total_setores': arquivo_df['CD_SETOR'].nunique(),
    'media_consumo_diario_geral': arquivo_df['NN_CONSUMO_DIARIO'].mean(),
    'percentual_ambientalistas': (arquivo_df['TP_COMPORTAMENTO'] == 'AMBIENTALISTA').mean() * 100,
    'percentual_moderados': (arquivo_df['TP_COMPORTAMENTO'] == 'MODERADO').mean() * 100,
    'percentual_perdularios': (arquivo_df['TP_COMPORTAMENTO'] == 'PERDULARIO').mean() * 100
}

resumo_geral_df = pd.DataFrame([resumo_geral])
caminho_resumo_geral = os.path.join(caminho, 'includes\\Resumo_Geral_Comportamento.csv')
resumo_geral_df.to_csv(caminho_resumo_geral, sep=';', index=False, float_format='%.2f')

print("\nRelatórios salvos com sucesso:")
print(f"- Análise por setor: {caminho_relatorio}")
print(f"- Resumo geral: {caminho_resumo_geral}")


Verificando dados nulos:
SK_MATRICULA               0
NM_LOCALIDADE              0
NM_CATEGORIATARIFARIA      0
NM_SITUACAO_IMOVEL         0
NN_MORADORES               0
ST_PISCINA                 0
LAT_GEO                    0
LONG_GEO                   0
X                          0
Y                          0
CD_SETOR                   0
NN_MEDIA_CONSUMO         527
NN_CONSUMO_DIARIO        527
TP_COMPORTAMENTO           0
dtype: int64

Top 5 setores com mais perdulários:
   CD_SETOR  total_consumidores  ambientalistas  moderados  perdularios  \
83     S302                   1               0          0            1   
95     S325                   1               0          0            1   
33     S107                 226              15         15          196   
34     S108                 222              34         26          162   
50     S168                  91              21          4           66   

    media_consumo_diario  %_ambientalistas  %_moderados  %_perdular